In [20]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import csv

from helpers import bert_helper

In [21]:
### Load pretrained BERT model 

In [22]:
#!pip install pytorch-pretrained-bert

import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
#% matplotlib inline


# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpzm4uetdw
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

In [23]:
# try it for one set of tokens

In [33]:

word = "water"
tokens = [] 

fieldnames = ['word', 'sentence', 'pos', 'bnc_index']
with open('../data/word_data/water/BNC_tokens.csv', mode="r") as infile:
    reader = csv.DictReader(infile, delimiter='\t', fieldnames=fieldnames)
    
    tokens = [row for row in reader]
    #print(tokens)
        
    print("number of data points: %s" % len(tokens))              
    #data = pd.DataFrame.from_records(tokens, columns=["word", "sentence", "pos", "bnc_index"])
    print('Size of the dataframe: {}'.format(data.shape))


K = 10
layers = layers = [0,1,2,3,4,5,5,7,8,9,10,11]

# get entire activation
for row in tokens:
    sentence = row["sentence"]
    vector = bert_helper.get_bert_vectors_for(word, sentence, model, tokenizer)
    # if the token was too long we may not have succeeded in generating embeddings for it, in which case we will throw it out
    if vector != None:
        row["embedding"] = vector
    else:
        row["embedding"] = None

data = list(filter(lambda row: row['embedding'] != None, data))

for layer in layers:
    layer_vectors = [row['embedding'][layer] for row in data]

    # AT EACH LAYER, cluster with k=10
    # calculate clusters
    kmeans_obj = KMeans(n_clusters=10)
    kmeans_obj.fit(layer_vectors)
    label_list = kmeans_obj.labels_
    cluster_centroids = kmeans_obj.cluster_centers_


    # store clusternumber with data
    for index,datapoint in enumerate(data):
        datapoint['cluster_number'] = label_list[index]


    # retrieve centroid for each cluster and uids of sentences in cluster:
    for clusternumber in range(10):
        centroid = cluster_centroids[clusternumber]
        sentence_uids = []

        cluster_vectors = []
        # find the tokens in the dataset that are in this cluster
        for index, datapoint in enumerate(data):
            if datapoint['cluster_number'] == clusternumber:
                sentence_uids.append(datapoint['uid'])
                cluster_vectors.append(layer_vectors[index])



        # calculate variance for this cluster
        # the sentence uids are the ones in this cluster. you need to get the vectors for them 
        # and calculate variance.
        cluster_variance = np.var(cluster_vectors)

        covariance = np.cov(cluster_vectors, rowvar=False)
        total_varianve = np.linalg.det(covariance)

        out_data = {'word': word,
                    'clusternumber': clusternumber,
                    'centroid': centroid,
                    'sentence_uids': sentence_uids,
                    'variance': cluster_variance,
                    'total_variance': total_variance,
                    'concreteness': ratings[word]['concreteness']
                   }

        # store in file
        # write dta for this cluster
        #writer.writerow(out_data)
        print(outdata)


    # save the token ids with the clustervector




number of data points: 101
Size of the dataframe: (101, 4)


TypeError: string indices must be integers